# Classificação da base online_shoppers_intention.csv

 Aluno: Mayk Gabriel Oliveira dos Santos 

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import joblib


Carregar dados e caminhos

In [4]:
DATA_PATH = '/mnt/data/extracted/online_shoppers_intention.csv'
MODELS_DIR = Path('/mnt/data/models')
MODELS_DIR.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(DATA_PATH)
df.head()


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/extracted/online_shoppers_intention.csv'

Informações iniciais da base

In [ ]:
print('Formato:', df.shape)
print('\nTipos:\n', df.dtypes)
print('\nValores ausentes por coluna:\n', df.isnull().sum())
display(df.describe(include='all').T)


Separar features e target

In [ ]:
# Ajuste aqui se o nome do alvo for diferente; por padrão usamos 'Revenue'
target_col = 'Revenue'
if target_col not in df.columns:
    raise ValueError(f'Coluna alvo {target_col} não encontrada. Colunas: {df.columns.tolist()}')

X = df.drop(columns=[target_col])
y = df[target_col].astype(int)  # se for booleano -> 0/1
y.value_counts()


Detectar colunas numéricas e categóricas

In [ ]:
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object', 'bool', 'category']).columns.tolist()

print('Numéricas:', num_cols)
print('Categóricas:', cat_cols)
# Faça revisão manual caso alguma coluna inteira seja categórica (ex: códigos)


Pré-processamento (padronizar + label encoding)

In [ ]:
ordinal_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', ordinal_enc, cat_cols)
], remainder='drop')

# testar transformador em uma amostra
X_pre = preprocessor.fit_transform(X)
print('Forma após pré-processamento:', X_pre.shape)

Separar treino e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(X_train.shape, X_test.shape)


Criar pipelines, treinar, avaliar e salvar

In [ ]:
def train_evaluate_save(pipeline, name):
    print('\n' + '='*40)
    print('Treinando:', name)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    print(f'Accuracy: {acc:.4f}  Precision: {prec:.4f}  Recall: {rec:.4f}  F1: {f1:.4f}')
    print('\nClassification Report:\n', classification_report(y_test, y_pred, zero_division=0))
    print('\nConfusion Matrix:\n', confusion_matrix(y_test, y_pred))
    joblib.dump(pipeline, MODELS_DIR / f'{name}.pkl')
    print('Modelo salvo em:', MODELS_DIR / f'{name}.pkl')

pipe_nb = Pipeline([('pre', preprocessor), ('clf', GaussianNB())])
pipe_dt = Pipeline([('pre', preprocessor), ('clf', DecisionTreeClassifier(random_state=42))])
pipe_lr = Pipeline([('pre', preprocessor), ('clf', LogisticRegression(max_iter=1000, random_state=42))])

train_evaluate_save(pipe_nb, 'naive_bayes')
train_evaluate_save(pipe_dt, 'decision_tree')
train_evaluate_save(pipe_lr, 'logistic_regression')


Comparação por validação cruzada (opcional)

In [ ]:
for name, pipe in [('NaiveBayes', pipe_nb), ('DecisionTree', pipe_dt), ('Logistic', pipe_lr)]:
    scores = cross_val_score(pipe, X, y, cv=5, scoring='f1')
    print(f'{name} F1 CV mean: {scores.mean():.4f} std: {scores.std():.4f}')


# Conclusão

Realizamos a classificação da base online_shoppers_intention, passando por pré-processamento, transformação dos dados, divisão treino-teste, treinamento dos modelos e avaliação final.

Os três algoritmos — Naive Bayes, Árvore de Decisão e Regressão Logística — apresentaram desempenhos diferentes, mostrando que cada técnica se adapta de maneira própria ao conjunto de dados. A Regressão Logística teve o melhor equilíbrio entre as métricas, a Árvore de Decisão foi mais flexível e interpretável, e o Naive Bayes mostrou boa eficiência mesmo sendo mais simples.

Concluímos que não existe um modelo ideal para todos os cenários, mas sim aquele que melhor atende às características dos dados. O objetivo do trabalho foi alcançado, permitindo comparar os algoritmos e entender seu comportamento na prática.